In [1]:
from importlib import reload
import pandas as pd
import numpy as np
from spektral.data import Dataset, Graph
import dataset
reload(dataset)
import pickle
import os

In [2]:
# Load combined data
df_raw = pd.read_csv('../data/combined.csv')
df = df_raw.copy()

# Load hero feature data
df_features = pd.read_csv('../data/features.csv')
df_features = df_features.set_index('hero_id')

# Load standard filter
df_filter = pd.read_csv('../models/filter.csv')

In [7]:
# Load graph dataset 50000 matches at a time
dir = '../data/graphs_v2/'
count = 0
total = len(df)
step = 50000

#for i in range(0,int(np.ceil(total/step))):
for i in range(0,int(1000000/step)):
    start = i*step
    end = start+step-1 if (start+step)<total else total-1
    path = dir+f'graphs_v2_{start}-{end}.pkl'
    print(path)
    file = open(path,'rb')
    if i==0:
        graphs = pickle.load(file)
    else:
        graphs = graphs + pickle.load(file)

../data/graphs_v2/graphs_v2_0-49999.pkl
../data/graphs_v2/graphs_v2_50000-99999.pkl
../data/graphs_v2/graphs_v2_100000-149999.pkl
../data/graphs_v2/graphs_v2_150000-199999.pkl
../data/graphs_v2/graphs_v2_200000-249999.pkl
../data/graphs_v2/graphs_v2_250000-299999.pkl
../data/graphs_v2/graphs_v2_300000-349999.pkl
../data/graphs_v2/graphs_v2_350000-399999.pkl
../data/graphs_v2/graphs_v2_400000-449999.pkl
../data/graphs_v2/graphs_v2_450000-499999.pkl
../data/graphs_v2/graphs_v2_500000-549999.pkl
../data/graphs_v2/graphs_v2_550000-599999.pkl
../data/graphs_v2/graphs_v2_600000-649999.pkl
../data/graphs_v2/graphs_v2_650000-699999.pkl
../data/graphs_v2/graphs_v2_700000-749999.pkl
../data/graphs_v2/graphs_v2_750000-799999.pkl
../data/graphs_v2/graphs_v2_800000-849999.pkl
../data/graphs_v2/graphs_v2_850000-899999.pkl
../data/graphs_v2/graphs_v2_900000-949999.pkl
../data/graphs_v2/graphs_v2_950000-999999.pkl


In [25]:
# Filter graph dataset
# NB: filtering will need some logic to deal with DotaV1 data - 2 graphs for every match
df_filter_sub = df_filter.iloc[0:1000000]
filter_values_sub = df_filter_sub['filter'].values
# Get indices of True filter values, as selectors for the Graph Dataset
filter_idx_sub = [i for i, x in enumerate(filter_values_sub) if x]
graphs_filt = graphs[filter_idx_sub]

In [27]:
graphs_filt

DotaV2(n_graphs=905542)

In [60]:
# MinMax Scaler model to normalise features from 0-1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df_features.iloc[:,3:].to_numpy())

MinMaxScaler()

In [ ]:
# Loop through each graph and scale feature matrix
print('Scaling graph dataset feature matrices:')
for i in range(0,len(graphs_filt)):
    if(i%100000==0):
        print(i)
    graphs_filt[i].x = scaler.transform(graphs_filt[i].x)

In [61]:
graphs_filt[4].x.shape

(10, 20)

In [62]:
x_new = scaler.transform(graphs_filt[4].x)

In [63]:
x_new.shape

(10, 20)